In [ ]:
import os
import subprocess
import optparse

import matplotlib
matplotlib.use("AGG")
import matplotlib.pyplot as plt
import numpy as np
import os

from obspy.core import UTCDateTime, Stream, AttribDict
from obspy import read_inventory
from obspy.clients.filesystem import sds
from obspy.signal.trigger import coincidence_trigger


In [ ]:
sds_root = r"D:\DATA\merapi\data_sds"
response_root = r"D:\DATA\merapi\merapi_stationxml.xml"

In [ ]:
def getParameters(network):
    par = AttribDict()
    par.network = network
    par.filter = AttribDict(type="bandpass", freqmin=1.0, freqmax=20.0,
                            corners=2, zerophase=True)
    trace_ids = {}
    if network == "XM":
        trace_ids = {"XM.GRW0..BHZ": 1, "XM.GRW0..BHN": 1,
                     "XM.GRW0..BHE": 1}
        coinc_sum = 2.0
    elif network == "XM+":
        trace_ids = {"XM.GRW0..BHZ": 1, "XS.KLT0..BHZ": 1}
        coinc_sum = 2.0

    par.trace_ids = trace_ids
    # length of sta and lta in seconds
    par.coincidence = AttribDict(trigger_type="recstalta", sta=0.5, lta=10,
                                 thr_on=4.5, thr_off=0.5,
                                 thr_coincidence_sum=coinc_sum,
                                 trace_ids=trace_ids, max_trigger_length=120,
                                 trigger_off_extension=20)
    par.dir = "./XM_trigger"
    os.makedirs("%s"%par.dir, exist_ok=True)
    par.logfile = os.path.join(par.dir, "%s_log.txt" % network)
    par.trigfile = os.path.join(par.dir, "%s_trigger.txt" % network)
    return par


In [ ]:
def run(time, par):
    client = sds.Client(sds_root=sds_root)
    t1 = time
    t2 = t1 + (60 * 60 * 1) + 10
    st = Stream()
    num_stations = 0
    station_id = []
    possible_coinc_sum = 0
    exceptions = []
    inv = read_inventory(response_root)
    for trace_id, weight in par.trace_ids.items():
        print(trace_id)
        net, sta, loc, cha = trace_id.split(".")
        station_id.append("%s.%s.%s.%s*"%(net,sta,loc,cha[:-1]))

        for comp in "ZNE123":
            cha_ = cha[:-1] + comp
            try:
                # we request 60s more at start and end and cut them off later to
                # avoid a false trigger due to the tapering during instrument
                # correction
                tmp = client.get_waveforms(network=net, station=sta, location=loc, channel=cha_, starttime=t1 - 180, endtime=t2 + 180)
                tmp.attach_response(inv)
            except:
                exceptions.append("%s-%s" % (sta,cha_))
                continue
            if comp == cha[-1]:
                possible_coinc_sum += weight
            st.extend(tmp)
        num_stations += 1
    st.merge(-1)
    st.sort()

    trigger = []
    summary = []
    summary.append("#" * 79)
    summary.append("######## %s  ---  %s ########" % (t1, t2))
    summary.append("#" * 79)
    summary.append("######## %s  ---  %s ########" % (t1, t2))
    summary.append("#" * 79)
    summary.append(st.__str__(extended=True))
    if exceptions:
        summary.append("#" * 33 + " Exceptions  " + "#" * 33)
        summary += exceptions
    summary.append("#" * 79)

    st.traces = [tr for tr in st if tr.stats.npts > 1]

    trig = []
    mutt = []
    if st:
        # preprocessing, backup original data for plotting at end
        st.detrend("linear")
        st.merge(method=1, fill_value=0)
        for tr in st:
            perc = 1.0 / (tr.stats.endtime - tr.stats.starttime)
            perc = min(perc, 1)
            tr.taper(type="cosine", max_percentage=perc)
            print(tr)
            tr.remove_sensitivity()

        st.sort()
        st_trigger = st.copy()
        # prepare plotting
        st.trim(t1, t2, pad=True, fill_value=0)
        # triggering
        st_trigger.filter(**par.filter)
        # do the triggering (with additional data at sides to avoid artifacts
        trig = coincidence_trigger(stream=st_trigger, details=True,
                                  **par.coincidence)
        # restrict trigger list to time span of interest
        trig = [t for t in trig if (t1 <= t['time'] <= t2)]

        for t in trig:
            max_similarity = max(list(t['similarity'].values()) + [0])
            time_str = str(t['time']).split(".")[0]
            sta_string = "-".join(t['stations'])
            info = "%s %ss %s %.2f %s"
            info = info % (time_str, ("%.1f" % t['duration']).rjust(4),
                           ("%i" % t['cft_peak_wmean']).rjust(3),
                           max_similarity, sta_string)
            summary.append(info)
            sta_string = ",".join(station_id)
            if t['duration'] < 60:
                duration = 60.
            else:
                duration = t['duration']

            info = "%s %s %s"%\
                    (par.network, str(t['time'] - 20), str(duration+20))
            summary.append(info)
            trigger.append(info)
            # tmp = st.slice(t['time'] - 10, t['time'] + duration)
            # filename = "%s_%.1f_%i_%s-%s_%.2f_%s.png"
            # filename = filename % (time_str, t['duration'],
            #                        t['cft_peak_wmean'], t['coincidence_sum'],
            #                        possible_coinc_sum, max_similarity,
            #                        sta_string)
            # filename = os.path.join(par.dir, filename)
            # dpi = 72
            # fig = plt.figure(figsize=(700.0 / dpi, 400.0 / dpi))
            # for comp, color in zip("ENZ123", "rrkrrk"):
            #     try:
            #         tmp_ = tmp.select(component=comp).copy()
            #         tmp_.detrend("constant")
            #         tmp_.plot(fig=fig, color=color, number_of_ticks=6,
            #               linewidth=1.2, type="relative")
            #     except:
            #         continue;
            # for ax in fig.axes:
            #     ylims = ax.get_ylim()
            #     ax.set_yticks(ylims)
            #     ax.set_yticklabels(["%.1e" % (val * 1e3) for val in ylims])
            #     ax.set_ylabel("mm/s")
            # for ax in fig.axes[::2]:
            #     ax.yaxis.set_ticks_position("left")
            # for ax in fig.axes[1::2]:
            #     ax.yaxis.set_ticks_position("right")
            # for ax in fig.axes[:-1]:
            #     ax.set_xticks([])
            # try:
            #     fig.tight_layout()
            # except:
            #     pass
            # fig.subplots_adjust(hspace=0)
            # print(filename)
            # fig.savefig(filename.replace(":","_").replace("*","+"), dpi=dpi)
            # plt.close()
            # mutt += ("-a", filename)
        del tmp
        del st_trigger
        del tr
    del st

    summary.append("#" * 79)
    summary = "\n".join(summary)
    trigger = "\n".join(trigger)
    # avoid writing long list of streams when using many event templates
    par_tmp = par.copy()
    #for k, v in par_tmp.coincidence.event_templates.iteritems():
    #    par_tmp.coincidence.event_templates[k] = len(v)
    summary += "\n" + "\n".join(("%s=%s" % (k, v) for k, v in par_tmp.items()))
    #print summary
    with open(par.logfile, "wt") as fh:
        fh.write(summary + "\n")
    with open(par.trigfile, "at") as fu:
        fu.write(trigger + "\n")

    plt.close('all')
    del summary
    del trigger
    del mutt

                                                                        

In [ ]:
# setting the inputs

t1 = int(UTCDateTime(1998,7,4).timestamp)
t2 = int(UTCDateTime(1998,7,16).timestamp)
         
times = [UTCDateTime(t) for t in np.arange(t1, t2, 3600)]
# print(times)

network ="XM"
if network not in ("XM", "XM+"):
    print("Not the correct network")

par = getParameters(network)

fx = open(par.logfile, "w")
fx.close()


for t in times:
    run(t, par)
    # print(t)


